In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# SparkSession
URL_SPARK = "spark://spark-master:7077"

spark = (
    SparkSession.builder
    .appName("spark-ml")
    .config("executor.memory", "8g")
    .master(URL_SPARK)
    .getOrCreate()
)

23/05/31 03:13:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_avocado = spark.read.csv(
  "avocado.csv", 
  header=True, 
  inferSchema=True
)

# cache data
df_avocado.cache()
df_avocado.show(4)

+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|_c0|      Date|AveragePrice|Total Volume|   4046|     4225| 4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85|48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81|58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67|130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06|        1.08|    78992.15| 1132.0| 71976.41|72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
+---+----------+------------+------------

In [3]:
df_avocado_train, df_avocado_test = df_avocado.randomSplit([0.75, 0.25], seed=214)

In [4]:
df_avocado_train.show(2)

+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+----------------+
|_c0|      Date|AveragePrice|Total Volume|     4046|     4225|    4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|          region|
+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+----------------+
|  0|2015-12-27|        0.49|  1137707.43| 738314.8|286858.37|11642.46|  100891.8|  70749.02|  30142.78|        0.0|conventional|2015|   PhoenixTucson|
|  0|2015-12-27|        0.71|   776404.39|451904.51|141599.36|15486.97| 167413.55| 123158.22|  33065.33|    11190.0|conventional|2015|WestTexNewMexico|
+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+----------------+
only showing top 2 rows



In [5]:
import pyspark.ml

In [6]:
from pyspark.ml.feature import SQLTransformer
import numpy

COLUMNS = ['AveragePrice', 'type']
COLUMNS = [f"`{col}`" for col in COLUMNS]

LOG_COLUMNS =  ['4225', '4770', 'Small Bags', 'Large Bags', 'XLarge Bags']
LOG_COLUMNS = [f"LOG(`{col}`+1) AS `LOG {col}`" for col in LOG_COLUMNS]

sql_trans = SQLTransformer(
    statement=f"""
    SELECT
    {', '.join(COLUMNS)}
    , {', '.join(LOG_COLUMNS)}
    ,YEAR(__THIS__.Date)-2000 AS year
    ,MONTH(__THIS__.Date) AS month

    FROM __THIS__
    """
)

# Visualize the data
sql_trans.transform(df_avocado_train).show(4)

+------------+------------+------------------+------------------+------------------+------------------+-----------------+----+-----+
|AveragePrice|        type|          LOG 4225|          LOG 4770|    LOG Small Bags|    LOG Large Bags|  LOG XLarge Bags|year|month|
+------------+------------+------------------+------------------+------------------+------------------+-----------------+----+-----+
|        0.49|conventional|12.566747374652527| 9.362499927974252|11.166908098190957|10.313733879047971|              0.0|  15|   12|
|        0.71|conventional|11.860764002611406| 9.647818872531012| 11.72123326879331| 10.40627082310141|9.322865162818028|  15|   12|
|         0.8|conventional| 12.53017497505446|11.349393905288467|11.824526973139381| 9.415621332905047|9.658771095406955|  15|   12|
|         0.8|conventional|13.028501871764691|11.364461534887267|13.490872079413348| 11.21667384527801|9.342104328605496|  15|   12|
+------------+------------+------------------+------------------+----

In [7]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

# Creating a Month vector column
month_vec_ass = VectorAssembler(inputCols=['month'], outputCol='month_vec')

df_avocado_month_ass = month_vec_ass.transform(sql_trans.transform(df_avocado_train))

# Scaling the month column
month_scaler = MinMaxScaler(inputCol='month_vec', outputCol='month_scaled')
month_scaler = month_scaler.fit(df_avocado_month_ass)

month_scaler\
  .transform(df_avocado_month_ass)\
  .select( ['month', 'month_vec', 'month_scaled'] )\
  .show(10)

+-----+---------+------------+
|month|month_vec|month_scaled|
+-----+---------+------------+
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
|   12|   [12.0]|       [1.0]|
+-----+---------+------------+
only showing top 10 rows



In [8]:
from pyspark.ml.feature import StringIndexer
str_indexer = StringIndexer(inputCol="type", outputCol="type_index")

str_indexer = str_indexer.fit(df_avocado_train)

str_indexer\
  .transform(df_avocado_train)\
  .select( ["type", "type_index"] )\
  .show(4)

+------------+----------+
|        type|type_index|
+------------+----------+
|conventional|       0.0|
|conventional|       0.0|
|conventional|       0.0|
|conventional|       0.0|
+------------+----------+
only showing top 4 rows



In [9]:
# Apply transformations
## SQL transformer
df_avocado_train_transformed = sql_trans.transform(df_avocado_train)

## String indexer
df_avocado_train_transformed = str_indexer.transform(df_avocado_train_transformed)

## Month scaler (vector assembler + minmax scaler)
df_avocado_train_transformed = month_vec_ass.transform(df_avocado_train_transformed)
df_avocado_train_transformed = month_scaler.transform(df_avocado_train_transformed)


# Join all features into a single vector
numerical_vec_ass = VectorAssembler(
    inputCols=[
      'year', 'month_scaled', 'LOG 4225', 
      'LOG 4770', 'LOG Small Bags', 
      'LOG Large Bags', 'LOG XLarge Bags'
    ],
    outputCol='features_num'
)
df_avocado_train_transformed = numerical_vec_ass.transform(df_avocado_train_transformed)

# Join all categorical features into a single vector
categorical_vec_ass = VectorAssembler(
    inputCols=['type_index'],
    outputCol='features_cat'
)
df_avocado_train_transformed = categorical_vec_ass.transform(df_avocado_train_transformed)


# See the result
df_avocado_train_transformed.select(['features_cat', 'features_num', 'AveragePrice']).show(4, False)

+------------+-------------------------------------------------------------------------------------------------------+------------+
|features_cat|features_num                                                                                           |AveragePrice|
+------------+-------------------------------------------------------------------------------------------------------+------------+
|[0.0]       |[15.0,1.0,12.566747374652527,9.362499927974252,11.166908098190957,10.313733879047971,0.0]              |0.49        |
|[0.0]       |[15.0,1.0,11.860764002611406,9.647818872531012,11.72123326879331,10.40627082310141,9.322865162818028]  |0.71        |
|[0.0]       |[15.0,1.0,12.53017497505446,11.349393905288467,11.824526973139381,9.415621332905047,9.658771095406955] |0.8         |
|[0.0]       |[15.0,1.0,13.028501871764691,11.364461534887267,13.490872079413348,11.21667384527801,9.342104328605496]|0.8         |
+------------+--------------------------------------------------------------

In [10]:
from pyspark.ml.feature import StandardScaler

# Scaling the numerical features using a StandardScaler
std_scaler = StandardScaler(
    inputCol="features_num",
    outputCol="features_scaled",
    withStd=True,
    withMean=True
)

std_scaler = std_scaler.fit(df_avocado_train_transformed)
std_scaler.transform(df_avocado_train_transformed).select(['features_scaled']).show(5, False)

+----------------------------------------------------------------------------------------------------------------------------------------+
|features_scaled                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------+
|[-1.2177154955881637,1.6482225355667333,0.9527463109714546,1.0269649008115518,0.5657377199959452,0.8334134211814762,-0.6436162273445295]|
|[-1.2177154955881637,1.6482225355667333,0.7058305701685025,1.0954357394643428,0.7803295242390127,0.8574417380503548,2.012648481596976]  |
|[-1.2177154955881637,1.6482225355667333,0.9399552148956506,1.5037797059140563,0.8203168521795554,0.6002078289352569,2.1083545825302594] |
|[-1.2177154955881637,1.6482225355667333,1.1142436751287843,1.5073956355774096,1.4653967110976907,1.0678725104034048,2.0181300922626053] |
|[-1.2177154955881637,1.648

In [11]:
# Machine learning pipeline
from pyspark.ml import Pipeline

# Create a preprocessing pipeline
prepro_pipe = Pipeline(stages=[
    sql_trans,
    str_indexer,
    month_vec_ass,
    month_scaler,
    numerical_vec_ass,
    categorical_vec_ass,
    std_scaler,

    # Join all features into a single vector
    VectorAssembler(
        inputCols=['features_scaled', 'features_cat'],
        outputCol='features'
    ),
])


# Fit the pipeline
pipeline_model = prepro_pipe.fit(df_avocado_train)

# Transform the data
df_avocado_train_transformed = pipeline_model.transform(df_avocado_train)

# See the result
df_avocado_train_transformed.select(['features', 'AveragePrice']).show(4, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|features                                                                                                                                    |AveragePrice|
+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|[-1.2177154955881637,1.6482225355667333,0.9527463109714546,1.0269649008115518,0.5657377199959452,0.8334134211814762,-0.6436162273445295,0.0]|0.49        |
|[-1.2177154955881637,1.6482225355667333,0.7058305701685025,1.0954357394643428,0.7803295242390127,0.8574417380503548,2.012648481596976,0.0]  |0.71        |
|[-1.2177154955881637,1.6482225355667333,0.9399552148956506,1.5037797059140563,0.8203168521795554,0.6002078289352569,2.1083545825302594,0.0] |0.8         |
|[-1.2177154955881637,1.6482225355667333,1.1142436751287843,1.50

In [12]:
from pyspark.ml.regression import LinearRegression

# Create a linear regression model
lin_reg = LinearRegression(
    featuresCol='features',
    labelCol='AveragePrice',
    predictionCol='prediction',

    # Hyperaparameters
    maxIter=1000,
    regParam=0.3,       # Regularization
    elasticNetParam=0.8 # Regularization mixing parameter. 1 for L1, 0 for L2.
)

In [13]:
# Fit the model
lin_reg_model = lin_reg.fit(df_avocado_train_transformed)

# See the output
df_avocado_train_pred = lin_reg_model.transform(df_avocado_train_transformed)
df_avocado_train_pred.select(
  ['features', 'AveragePrice', 'prediction']
).show(4, False)

23/05/31 03:16:15 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/31 03:16:15 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+--------------------------------------------------------------------------------------------------------------------------------------------+------------+------------------+
|features                                                                                                                                    |AveragePrice|prediction        |
+--------------------------------------------------------------------------------------------------------------------------------------------+------------+------------------+
|[-1.2177154955881637,1.6482225355667333,0.9527463109714546,1.0269649008115518,0.5657377199959452,0.8334134211814762,-0.6436162273445295,0.0]|0.49        |1.4003505112793717|
|[-1.2177154955881637,1.6482225355667333,0.7058305701685025,1.0954357394643428,0.7803295242390127,0.8574417380503548,2.012648481596976,0.0]  |0.71        |1.4003505112793717|
|[-1.2177154955881637,1.6482225355667333,0.9399552148956506,1.5037797059140563,0.8203168521795554,0.6002078289352569,2.108354

In [14]:
# lin_reg_model.save('mymodel')

In [15]:
# from pyspark.ml.regression import LinearRegressionModel

# load_model = LinearRegressionModel.load('mymodel')

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator

reg_eval = RegressionEvaluator(
    labelCol='AveragePrice',
    predictionCol='prediction',
    metricName='rmse' # Root mean squared error
)

# Evaluate the model
reg_eval.evaluate(df_avocado_train_pred)

# Output >> 0.3978489578943717

0.3978489578943717

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


ml_pipeline = Pipeline(stages=[
    prepro_pipe, # Preprocessing pipeline
    lin_reg      # Linear regression model
])


param_grid = ParamGridBuilder() \
    .addGrid(lin_reg.regParam, [0.0, 0.1, 0.3, 0.5]) \
    .addGrid(lin_reg.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

In [ ]:
reg_eval = RegressionEvaluator(
    labelCol='AveragePrice',
    predictionCol='prediction',
    metricName='rmse' # Root mean squared error
)

# Join everything together using a CrossValidator object.
crossval_ml = CrossValidator(
    estimator=ml_pipeline, 
    estimatorParamMaps=param_grid, 
    evaluator=reg_eval, 
    numFolds=4
)

In [ ]:
crossval_ml_model = crossval_ml.fit(df_avocado_train)